# Data Wrangling and Preparation for Chicago Weather Forecasting

For this weather prediction project, we shall rely on historic hourly weather data available from the US Govenment. It is available publicly and free of change: https://www.ncdc.noaa.gov/cdo-web/datatools/lcd

The documentation, as provided by the US Government: https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/LCD_documentation.pdf

We shall be using 10 years of data (2011-2020) for both the Target Location (Chicago, IL) and for a number of nearby locations in the US Midwest in the states of Illinois, Missoury, Wisconsin, Minnesota, Michigan, Indiana and Ohio.

In [117]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv("../raw-data/noaa_2011-2020_chicago.csv", low_memory=False, parse_dates=['DATE'])
print(raw_df.columns.values)

['STATION' 'DATE' 'REPORT_TYPE' 'SOURCE' 'AWND' 'BackupDirection'
 'BackupDistance' 'BackupDistanceUnit' 'BackupElements' 'BackupElevation'
 'BackupElevationUnit' 'BackupEquipment' 'BackupLatitude'
 'BackupLongitude' 'BackupName' 'CDSD' 'CLDD' 'DSNW'
 'DailyAverageDewPointTemperature' 'DailyAverageDryBulbTemperature'
 'DailyAverageRelativeHumidity' 'DailyAverageSeaLevelPressure'
 'DailyAverageStationPressure' 'DailyAverageWetBulbTemperature'
 'DailyAverageWindSpeed' 'DailyCoolingDegreeDays'
 'DailyDepartureFromNormalAverageTemperature' 'DailyHeatingDegreeDays'
 'DailyMaximumDryBulbTemperature' 'DailyMinimumDryBulbTemperature'
 'DailyPeakWindDirection' 'DailyPeakWindSpeed' 'DailyPrecipitation'
 'DailySnowDepth' 'DailySnowfall' 'DailySustainedWindDirection'
 'DailySustainedWindSpeed' 'DailyWeather' 'HDSD' 'HTDD' 'HeavyFog'
 'HourlyAltimeterSetting' 'HourlyDewPointTemperature'
 'HourlyDryBulbTemperature' 'HourlyPrecipitation'
 'HourlyPresentWeatherType' 'HourlyPressureChange'
 'HourlyPres

We can see that most of these columns are not what we need (which is Hourly data). We also know that `REPORT_TYPE` of `SOD` indicates Daily Data in which we are not interested. Throw away the extra columns and the reports that are not hourly as follows:

In [118]:
raw_df = raw_df[raw_df['REPORT_TYPE'].str.strip() != 'SOD' ]
columns_to_drop = list(filter(lambda c: c != 'DATE' and not c.startswith('Hourly'), raw_df.columns.tolist()))
raw_df = raw_df.drop(columns = columns_to_drop)

print(raw_df.columns.values)

['DATE' 'HourlyAltimeterSetting' 'HourlyDewPointTemperature'
 'HourlyDryBulbTemperature' 'HourlyPrecipitation'
 'HourlyPresentWeatherType' 'HourlyPressureChange'
 'HourlyPressureTendency' 'HourlyRelativeHumidity'
 'HourlySeaLevelPressure' 'HourlySkyConditions' 'HourlyStationPressure'
 'HourlyVisibility' 'HourlyWetBulbTemperature' 'HourlyWindDirection'
 'HourlyWindGustSpeed' 'HourlyWindSpeed']


There, much more manageable!

Before we go ahead further, though, let's keep only the columns that we may need for future modeling. Based on basic knowledge of weather we can see that some of the data is mutually redundant:

- 3 kinds of pressure (keep `AltimeterSetting` only)
- 2 kinds of temperature (keep `DryBulb` only)
- 2 kinds of pressure change (keep `PressureChange` only)



In [119]:
raw_df = raw_df.drop(columns = ['HourlyPressureTendency', 'HourlySeaLevelPressure', 
                                'HourlyStationPressure', 'HourlyWetBulbTemperature'])

Let's take a look at the remaing data:

In [120]:
raw_df.describe(include = 'all', datetime_is_numeric=True)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
count,125865,111200,125696,125707,99058,30244,42934,125693.000000,115054,125714,125523,20952.000000,125662.000000
unique,NaN,209,114,136,179,386,77,NaN,26640,43,38,NaN,NaN
top,NaN,29.98,30,34,0.00,BR:1 ||,+0.01,NaN,CLR:00,10.00,000,NaN,NaN
freq,NaN,2347,2832,2871,73356,5454,5477,NaN,6877,79622,7072,NaN,NaN
mean,2016-01-10 12:40:13.828625664,NaN,NaN,NaN,NaN,NaN,NaN,70.866063,NaN,NaN,NaN,26.545294,9.917119
min,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,14.000000,0.000000
25%,2013-07-24 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,59.000000,NaN,NaN,NaN,22.000000,6.000000
50%,2016-01-17 23:51:00,NaN,NaN,NaN,NaN,NaN,NaN,73.000000,NaN,NaN,NaN,25.000000,9.000000
75%,2018-07-06 14:51:00,NaN,NaN,NaN,NaN,NaN,NaN,85.000000,NaN,NaN,NaN,30.000000,13.000000
max,2020-12-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,70.000000,55.000000


We can see that the stats are incomplete due to the abundance of missing values. Let's take a look at some rows:

In [121]:
pd.set_option('display.max_rows', 100)
raw_df.head(20)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2011-01-01 00:00:00,NaN,35,42,NaN,NaN,-0.04,76.0,NaN,9.94,260,26.0,15.0
1,2011-01-01 00:49:00,29.72,30,39,NaN,NaN,NaN,70.0,BKN:07 26 OVC:08 37,10.00,250,NaN,11.0
2,2011-01-01 00:51:00,29.72,30,40,0.00,NaN,NaN,68.0,BKN:07 26 OVC:08 37,10.00,240,NaN,13.0
3,2011-01-01 01:18:00,29.73,28,37,NaN,NaN,NaN,70.0,OVC:08 30,10.00,260,NaN,17.0
4,2011-01-01 01:51:00,29.74,28,37,0.00,NaN,NaN,70.0,BKN:07 32,10.00,260,22.0,17.0
5,2011-01-01 02:51:00,29.75,27,36,0.00,NaN,-0.02,70.0,OVC:08 43,10.00,240,24.0,17.0
6,2011-01-01 03:51:00,29.75,20,32,0.00,NaN,NaN,61.0,BKN:07 48,10.00,240,25.0,15.0
7,2011-01-01 04:51:00,29.76,19,31,0.00,NaN,NaN,61.0,SCT:04 48,10.00,240,25.0,16.0
8,2011-01-01 05:51:00,29.77,17,28,0.00,NaN,-0.02,63.0,FEW:02 48,10.00,240,28.0,18.0
9,2011-01-01 06:00:00,NaN,17,28,NaN,NaN,-0.02,63.0,NaN,9.94,240,28.0,18.0


Yes, there are quite a few NANs as well as funny characters in Numeric columns.

Also, pay particular attention at `HourlySkyConditions` and `PresentWeatherType`. These are some funky codes that need to be made sense of for the data to be usable!

Much tweaking and imputation is needed to clean this mess up. 

We shall do that step by step:

In [122]:
# Precipitation: replace 'T' ("trace amount") with a small value
raw_df.loc[raw_df['HourlyPrecipitation'].str.strip() == 'T', 'HourlyPrecipitation'] = 0.005


In [123]:
#WindDirection: NAN-ify VRB values (aka "Variable Wind")
raw_df.loc[raw_df['HourlyWindDirection'].str.strip() == 'VRB', 'HourlyWindDirection'] = ""

In [124]:
# Put NANs in for variables that are meant to be Numeric but aren't
def removeJunkSuffixesAndPrefixes(df, variables):
    for variable in variables:
        if df[variable].dtypes != np.float64:
            df.loc[df[variable].str.contains('s', na=False), variable] = None
            df.loc[df[variable].str.contains('V', na=False), variable] = None
            df.loc[df[variable].str.contains('\\*', na=False), variable] = None

    return df

numeric_variables = ['HourlyDryBulbTemperature', 'HourlyRelativeHumidity', 'HourlyWindSpeed', 'HourlyWindDirection', 'HourlyDewPointTemperature', 
    'HourlyWindGustSpeed', 'HourlyAltimeterSetting', 'HourlyPressureChange', 'HourlyPrecipitation', 'HourlyPressureChange', 'HourlyVisibility']
raw_df = removeJunkSuffixesAndPrefixes(raw_df, numeric_variables)

In [125]:
# WindGust: replace NANs with 0 (assume no WindGust if not reported)
raw_df['HourlyWindGustSpeed'] = raw_df['HourlyWindGustSpeed'].fillna(0)

In [126]:
# Weather Type: decipher the finicky Government Terminology (explained in the PDF)
def preprocWeatherType(weatherType):
    weatherType = str(weatherType).strip()
    thunder_suffix = ""

    # "Weather" refers to the presense of water or other types of particles obscuring visibility
    # If not reported, that means that the conditions are clear
    if weatherType == "nan":
        return "NoPrecipitation"

    if "TS" in weatherType:
        thunder_suffix = "Thunder"

    if "SN" in weatherType:
        if "-SN" in weatherType:
            return "LightSnow" + thunder_suffix
        elif("+SN" in weatherType):
            return "HeavySnow" + thunder_suffix
        else:
            return "Snow" + thunder_suffix

    if "RA" in weatherType:
        if "-RA" in weatherType:
            return "LightRain" + thunder_suffix
        elif("+RA" in weatherType):
            return "HeavyRain" + thunder_suffix
        else:
            return "Rain" + thunder_suffix
    if "DZ" in weatherType:
        return "LightRain"

    if "UP" in weatherType:
        return "UnknownPrecipitation" + thunder_suffix

    if "BR" in weatherType:
        return "Mist" + thunder_suffix

    if "FG" in weatherType:
        return "Fog" + thunder_suffix
    if "HZ" in weatherType or "FU" in weatherType:
        return "Haze" + thunder_suffix
    if "PL" in weatherType or "GR" in weatherType:
        return "Hail"+thunder_suffix
    if "SQ" in weatherType: #Squall
        return thunder_suffix

    if thunder_suffix != "":
        return thunder_suffix

    return None

raw_df['HourlyPresentWeatherType'] = raw_df['HourlyPresentWeatherType'].apply(preprocWeatherType)


In [127]:
# Sky Condition: decipher the finicky Government Terminology (explained in the PDF)
#...but first, impute missing values by FF/BF filling (assuming weather changes gradually from observation to observation)
raw_df['HourlySkyConditions'] = raw_df['HourlySkyConditions'].fillna(method="ffill")
raw_df['HourlySkyConditions'] = raw_df['HourlySkyConditions'].fillna(method="bfill")

coverCodes = {
    "CLR" : "Clear",
    "FEW" : "MostlyClear",
    "SCT" : "PartlyCloudy",
    "BKN" : "MostlyCloudy",
    "OVC" : "Cloudy"
}
def preprocSkyConditions(skyCondition):
    skyCondition = str(skyCondition).strip()
    if skyCondition == "nan":
        return None, 0

# Format Example: BKN:07 6 BKN:07 100 BKN:07 120
# Sky Condition is reported in 3 layers; as per data set guidance, consider only the 3rd layer to summarize overall condition
#   The 3-letter code is the actual condition in Aviation Terminology
#   The :XX that follows is its numeric representation and can be dropped
# The final number is the cloud layer's height above ground, in hundreds of feet

    last_colon_index = skyCondition.rfind(":")
    if last_colon_index == -1:
        return "Clear", 0

    if skyCondition[last_colon_index-1] == 'V':
        cover = "Obscured"
    else:
        try:
            cover = coverCodes[skyCondition[last_colon_index-3 : last_colon_index]]
        except:
            cover = "Obscured"

    height = 0
    if cover != 'Clear':
        try:
            height = int(skyCondition[last_colon_index+3 :]) * 100
        except ValueError:
            height = 1
    else:
        height = 30000 #If it's clear, assume clouds are very very high

    return cover, height

raw_df['HourlySkyConditions'], raw_df['HourlyCloudHeight'] = zip(*raw_df['HourlySkyConditions'].apply(preprocSkyConditions))

# Let's take a peek at the dataset now
raw_df.head(20)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,HourlyCloudHeight
0,2011-01-01 00:00:00,NaN,35,42,NaN,NoPrecipitation,-0.04,76.0,Cloudy,9.94,260,26.0,15.0,3700
1,2011-01-01 00:49:00,29.72,30,39,NaN,NoPrecipitation,NaN,70.0,Cloudy,10.00,250,0.0,11.0,3700
2,2011-01-01 00:51:00,29.72,30,40,0.00,NoPrecipitation,NaN,68.0,Cloudy,10.00,240,0.0,13.0,3700
3,2011-01-01 01:18:00,29.73,28,37,NaN,NoPrecipitation,NaN,70.0,Cloudy,10.00,260,0.0,17.0,3000
4,2011-01-01 01:51:00,29.74,28,37,0.00,NoPrecipitation,NaN,70.0,MostlyCloudy,10.00,260,22.0,17.0,3200
5,2011-01-01 02:51:00,29.75,27,36,0.00,NoPrecipitation,-0.02,70.0,Cloudy,10.00,240,24.0,17.0,4300
6,2011-01-01 03:51:00,29.75,20,32,0.00,NoPrecipitation,NaN,61.0,MostlyCloudy,10.00,240,25.0,15.0,4800
7,2011-01-01 04:51:00,29.76,19,31,0.00,NoPrecipitation,NaN,61.0,PartlyCloudy,10.00,240,25.0,16.0,4800
8,2011-01-01 05:51:00,29.77,17,28,0.00,NoPrecipitation,-0.02,63.0,MostlyClear,10.00,240,28.0,18.0,4800
9,2011-01-01 06:00:00,NaN,17,28,NaN,NoPrecipitation,-0.02,63.0,MostlyClear,9.94,240,28.0,18.0,4800


In [128]:
# Now that all columns contain either clean numeric, categoric or NAN data, impute the NANs
#  (NOTE: weather overtime is a continuous process so impute by Forward/Backfilling)
raw_df = raw_df.fillna(method="ffill")
raw_df = raw_df.fillna(method="bfill")

Let's take a look at what the data looks like now:

In [129]:
raw_df.head(20)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,HourlyCloudHeight
0,2011-01-01 00:00:00,29.72,35,42,0.00,NoPrecipitation,-0.04,76.0,Cloudy,9.94,260,26.0,15.0,3700
1,2011-01-01 00:49:00,29.72,30,39,0.00,NoPrecipitation,-0.04,70.0,Cloudy,10.00,250,0.0,11.0,3700
2,2011-01-01 00:51:00,29.72,30,40,0.00,NoPrecipitation,-0.04,68.0,Cloudy,10.00,240,0.0,13.0,3700
3,2011-01-01 01:18:00,29.73,28,37,0.00,NoPrecipitation,-0.04,70.0,Cloudy,10.00,260,0.0,17.0,3000
4,2011-01-01 01:51:00,29.74,28,37,0.00,NoPrecipitation,-0.04,70.0,MostlyCloudy,10.00,260,22.0,17.0,3200
5,2011-01-01 02:51:00,29.75,27,36,0.00,NoPrecipitation,-0.02,70.0,Cloudy,10.00,240,24.0,17.0,4300
6,2011-01-01 03:51:00,29.75,20,32,0.00,NoPrecipitation,-0.02,61.0,MostlyCloudy,10.00,240,25.0,15.0,4800
7,2011-01-01 04:51:00,29.76,19,31,0.00,NoPrecipitation,-0.02,61.0,PartlyCloudy,10.00,240,25.0,16.0,4800
8,2011-01-01 05:51:00,29.77,17,28,0.00,NoPrecipitation,-0.02,63.0,MostlyClear,10.00,240,28.0,18.0,4800
9,2011-01-01 06:00:00,29.77,17,28,0.00,NoPrecipitation,-0.02,63.0,MostlyClear,9.94,240,28.0,18.0,4800


Wow, this looks much clearer now and is readable to a layperson! Let's also show some stats:

In [130]:
# Force all numeric columns to be numeric
raw_df[numeric_variables] = raw_df[numeric_variables].apply(pd.to_numeric)

raw_df.describe(include = 'all', datetime_is_numeric=True)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,HourlyCloudHeight
count,125865,125865.000000,125865.000000,125865.000000,125865.000000,125865,125865.000000,125865.000000,125865,125865.000000,124148.000000,125865.000000,125865.000000,125865.000000
unique,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NoPrecipitation,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,95621,NaN,NaN,53677,NaN,NaN,NaN,NaN,NaN
mean,2016-01-10 12:40:13.828625664,29.991843,40.171001,50.310340,0.009069,NaN,0.002282,70.873603,NaN,8.554360,182.514177,4.418838,9.914512,12919.522234
min,2011-01-01 00:00:00,29.030000,-32.000000,-23.000000,0.000000,NaN,-0.280000,12.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013-07-24 06:00:00,29.860000,27.000000,34.000000,0.000000,NaN,-0.020000,59.000000,NaN,9.000000,80.000000,0.000000,6.000000,2600.000000
50%,2016-01-17 23:51:00,29.990000,40.000000,50.000000,0.000000,NaN,0.000000,73.000000,NaN,10.000000,200.000000,0.000000,9.000000,9000.000000
75%,2018-07-06 14:51:00,30.130000,56.000000,68.000000,0.000000,NaN,0.030000,85.000000,NaN,10.000000,270.000000,0.000000,13.000000,25000.000000
max,2020-12-31 23:59:00,30.880000,79.000000,103.000000,3.040000,NaN,0.300000,100.000000,NaN,10.000000,360.000000,70.000000,55.000000,35000.000000


The numbers above do look plausible so we are on the right track!

One more hurdle to clear is deciding what to do with the fact that there are often several rows corresponding to the same hour, spaced at irregular minute intervals. For modeling purposes, we would benefit from data points that are evenly spaced chronologically.

Use Pandas aggregation to roll the reports into 1h time slots applying different aggregation functions. *Mean()* is by far the most sensible one (think multiple temperature reports per hour) and will be used for most variables. See comments for exceptions to that rules. 

In [131]:
# Sky conditions are categorical and need to be aggregated in a custom fashion
def aggSkyCondition(series):
    # Err on the side of the most clouded sky; taking the most pessimistic view is advantageous for forecasting
    #  (e.g. if it was Partly Cloudy one time during the hour and  Cloudy the other time, consider the hour Cloudy)
    priorities = ['Cloudy', 'MostlyCloudy', 'PartlyCloudy', 'MostlyClear', 'Clear']
    series = series.values
    for priority in priorities:
        if priority in series:
            return priority

    return 'Obscured'

# Similarly, Weather Type (visibility Obsucration due to water/other particles in the air) needs custom aggregation
def aggWeatherType(series):
    series = series.values
    if len(series) < 1:
        return None

    # Err on the side of the most severe weather by taking a pessemistic view
    #  (e.g. if it rained and 12:30 but not at 12:55, consider the 12:00 observation to be Rainy)
    priorities = ["Heavy", "RainThunder", "Rain", "Snow", "Hail", "UnknownPrecipitation", "Thunder", "Mist", "Fog", "Haze"]
    for priority in priorities:
        for s in series:
            if priority in s:
                return s

    return series[0]

In [132]:

####################################################################################
# !!! ATTENTION !!!: 
#   The following defines the final output to the downstream Modeling process
####################################################################################
agg_df = raw_df.groupby(pd.Grouper(freq='1h', key="DATE")).agg(
    Temp = pd.NamedAgg(column="HourlyDryBulbTemperature", aggfunc="mean"),
    Humidity = pd.NamedAgg(column="HourlyRelativeHumidity", aggfunc="mean"),
    DewPoint = pd.NamedAgg(column="HourlyDewPointTemperature", aggfunc="mean"),
    WindSpeed = pd.NamedAgg(column="HourlyWindSpeed", aggfunc="mean"),
    WindGust = pd.NamedAgg(column="HourlyWindGustSpeed", aggfunc="max"), # Gusts are gusty- pick the highest to represent the hour
    WindDirection = pd.NamedAgg(column="HourlyWindDirection", aggfunc="mean"),
    Precipitation = pd.NamedAgg(column="HourlyPrecipitation", aggfunc="max"), # Precipitation is culumative for the hour
    CloudCondition = pd.NamedAgg(column='HourlySkyConditions', aggfunc=aggSkyCondition), #Custom Aggregation
    CloudAltitude = pd.NamedAgg(column="HourlyCloudHeight", aggfunc="mean"),
    WeatherType = pd.NamedAgg(column='HourlyPresentWeatherType', aggfunc=aggWeatherType), #Custom Aggregation
    Pressure = pd.NamedAgg(column="HourlyAltimeterSetting", aggfunc="mean"),
    PressureChange = pd.NamedAgg(column="HourlyPressureChange", aggfunc="mean"),
    Visibility = pd.NamedAgg(column="HourlyVisibility", aggfunc="mean"),
    )

# Impute any values that may still be NANs by adopting the "Weather is Continuous in time" philosophy
agg_df = agg_df.fillna(method="ffill")
agg_df = agg_df.fillna(method="bfill")

We are getting close to the final answer, let's take another look at the data, paying special attention at the timestamps that we expect to be regularly spaced at 1h intervals:

In [133]:
agg_df.head(20)

,Temp,Humidity,DewPoint,WindSpeed,WindGust,WindDirection,Precipitation,CloudCondition,CloudAltitude,WeatherType,Pressure,PressureChange,Visibility
DATE,,,,,,,,,,,,,
2011-01-01 00:00:00,40.333333,71.333333,31.666667,13.0,26.0,250.0,0.000,Cloudy,3700.0,NoPrecipitation,29.720,-0.04,9.98
2011-01-01 01:00:00,37.000000,70.000000,28.000000,17.0,22.0,260.0,0.000,Cloudy,3100.0,NoPrecipitation,29.735,-0.04,10.00
2011-01-01 02:00:00,36.000000,70.000000,27.000000,17.0,24.0,240.0,0.000,Cloudy,4300.0,NoPrecipitation,29.750,-0.02,10.00
2011-01-01 03:00:00,32.000000,61.000000,20.000000,15.0,25.0,240.0,0.000,MostlyCloudy,4800.0,NoPrecipitation,29.750,-0.02,10.00
2011-01-01 04:00:00,31.000000,61.000000,19.000000,16.0,25.0,240.0,0.000,PartlyCloudy,4800.0,NoPrecipitation,29.760,-0.02,10.00
2011-01-01 05:00:00,28.000000,63.000000,17.000000,18.0,28.0,240.0,0.000,MostlyClear,4800.0,NoPrecipitation,29.770,-0.02,10.00
2011-01-01 06:00:00,27.500000,67.500000,18.000000,17.0,28.0,240.0,0.000,MostlyCloudy,8400.0,NoPrecipitation,29.785,-0.02,9.97
2011-01-01 07:00:00,25.000000,75.000000,18.000000,20.0,28.0,240.0,0.000,MostlyCloudy,11000.0,NoPrecipitation,29.810,-0.02,10.00
2011-01-01 08:00:00,23.000000,65.000000,13.000000,21.0,30.0,240.0,0.000,Cloudy,10000.0,NoPrecipitation,29.870,-0.09,10.00


Since the purpose here is to feed the output file to Machine Learning algorithms, come up with some derived variables that are conducive to that end. Prefix those with `_` for readability.

In [134]:
# Derive a boolean indication of whether the condition is Clear (with Classification in mind)
#  as well as a numeric cloud intensity variable
conditionIntensityMap = {
    "Clear" : 0,
    "MostlyClear" : 1,
    "PartlyCloudy" : 2,
    "MostlyCloudy": 4,
    "Cloudy": 5,
    "Obscured": 5
}

def deriveSkyConditionVars(skyCondition, weatherType):
    is_clear = 0
    cloud_intensity = 0

    # We consider the weather "Clear" if there are few clouds and no mist or fog
    if (skyCondition == "Clear" or skyCondition == "MostlyClear" or skyCondition == "PartlyCloudy") and ('Fog' not in weatherType and 'Mist' not in weatherType):
        is_clear = 1

    cloud_intensity = conditionIntensityMap[skyCondition]

    return is_clear, cloud_intensity

# Derive boolean indications of types of precipitation (once again, with Classification in Mind)
def deriveWeatherTypeVars(weatherType):
    is_precip = 0
    is_thunder = 0
    is_snow = 0 
    is_mist = 0

    if ("Rain" in weatherType) or ("Snow" in weatherType) or ("Hail" in weatherType) or ("Thunder" in weatherType) or ("UnknownPrecipitation" in weatherType):
        is_precip = 1

    if "Thunder" in weatherType:
        is_thunder = 1

    if "Snow" in weatherType:
        is_snow = 1

    # According to online resources, mist and fog are different degrees of the same thing: droplets of water suspended in the air
    #	(and neither is considered "precipitation")
    if ("Mist" in weatherType) or ("Fog" in weatherType): 
        is_mist = 1


    return is_precip, is_thunder, is_snow, is_mist

In [135]:
# Summarize Sky Condition in non-categorical format
agg_df['_is_clear'], agg_df['_cloud_intensity'] = zip(*agg_df.apply(lambda row: deriveSkyConditionVars(row['CloudCondition'], row['WeatherType']), axis=1))

# Summarize Weather Type in non-categorical format
agg_df['_is_precip'], agg_df['_is_thunder'], agg_df['_is_snow'], agg_df['_is_mist'] = zip(*agg_df['WeatherType'].apply(deriveWeatherTypeVars))

# Day-of-Year and Hour-of-Day are best encoded as Sin/Cos to reflect their circular nature
agg_df = agg_df.reset_index()
agg_df['_day_sin'] = np.sin(2 * np.pi * agg_df['DATE'].dt.dayofyear / 365)
agg_df['_day_cos'] = np.cos(2 * np.pi * agg_df['DATE'].dt.dayofyear / 365)
agg_df['_hour_sin'] = np.sin(2 * np.pi * agg_df['DATE'].dt.hour / 24)
agg_df['_hour_cos'] = np.cos(2 * np.pi * agg_df['DATE'].dt.hour / 24)

# Wind Direction is given as an angle: also encode that as sine/cosine
agg_df['_wind_dir_sin'] = np.sin(2 * np.pi * agg_df['WindDirection'] / 360)
agg_df['_wind_dir_cos'] = np.cos(2 * np.pi * agg_df['WindDirection'] / 360)

pd.set_option('display.max_rows', 100)
agg_df.head(20)

,DATE,Temp,Humidity,DewPoint,WindSpeed,WindGust,WindDirection,Precipitation,CloudCondition,CloudAltitude,...,_is_precip,_is_thunder,_is_snow,_is_mist,_day_sin,_day_cos,_hour_sin,_hour_cos,_wind_dir_sin,_wind_dir_cos
0,2011-01-01 00:00:00,40.333333,71.333333,31.666667,13.0,26.0,250.0,0.000,Cloudy,3700.0,...,0,0,0,0,0.017213,0.999852,0.000000e+00,1.000000e+00,-0.939693,-3.420201e-01
1,2011-01-01 01:00:00,37.000000,70.000000,28.000000,17.0,22.0,260.0,0.000,Cloudy,3100.0,...,0,0,0,0,0.017213,0.999852,2.588190e-01,9.659258e-01,-0.984808,-1.736482e-01
2,2011-01-01 02:00:00,36.000000,70.000000,27.000000,17.0,24.0,240.0,0.000,Cloudy,4300.0,...,0,0,0,0,0.017213,0.999852,5.000000e-01,8.660254e-01,-0.866025,-5.000000e-01
3,2011-01-01 03:00:00,32.000000,61.000000,20.000000,15.0,25.0,240.0,0.000,MostlyCloudy,4800.0,...,0,0,0,0,0.017213,0.999852,7.071068e-01,7.071068e-01,-0.866025,-5.000000e-01
4,2011-01-01 04:00:00,31.000000,61.000000,19.000000,16.0,25.0,240.0,0.000,PartlyCloudy,4800.0,...,0,0,0,0,0.017213,0.999852,8.660254e-01,5.000000e-01,-0.866025,-5.000000e-01
5,2011-01-01 05:00:00,28.000000,63.000000,17.000000,18.0,28.0,240.0,0.000,MostlyClear,4800.0,...,0,0,0,0,0.017213,0.999852,9.659258e-01,2.588190e-01,-0.866025,-5.000000e-01
6,2011-01-01 06:00:00,27.500000,67.500000,18.000000,17.0,28.0,240.0,0.000,MostlyCloudy,8400.0,...,0,0,0,0,0.017213,0.999852,1.000000e+00,6.123234e-17,-0.866025,-5.000000e-01
7,2011-01-01 07:00:00,25.000000,75.000000,18.000000,20.0,28.0,240.0,0.000,MostlyCloudy,11000.0,...,0,0,0,0,0.017213,0.999852,9.659258e-01,-2.588190e-01,-0.866025,-5.000000e-01
8,2011-01-01 08:00:00,23.000000,65.000000,13.000000,21.0,30.0,240.0,0.000,Cloudy,10000.0,...,0,0,0,0,0.017213,0.999852,8.660254e-01,-5.000000e-01,-0.866025,-5.000000e-01
9,2011-01-01 09:00:00,21.000000,62.000000,10.000000,23.0,32.0,250.0,0.000,Cloudy,20000.0,...,0,0,0,0,0.017213,0.999852,7.071068e-01,-7.071068e-01,-0.939693,-3.420201e-01


We are done with processing an individual dataset file; now let's do a little bit of sanity checking. 

Specifically, we wrote a lot of custom logic for clouds and precipitation decoding: let's make sure that the derived data makes sense with regard to that.

In [136]:
# (1) See if precipitation amounts and Weather Type correlate in a reasonable way
#  (e.g. we expect Rain precipitation to be more than Snow precipitation and No Precipitation to be less than the Snow one)
diag_df = agg_df.groupby('WeatherType').agg(
    AvgPrecip = pd.NamedAgg(column='Precipitation', aggfunc="mean"),
    Count = pd.NamedAgg(column='Precipitation', aggfunc="count")
    )
print(diag_df.sort_values(by='AvgPrecip'))

                      AvgPrecip  Count
WeatherType                           
                       0.000000      3
Fog                    0.000452    166
Haze                   0.000483    590
NoPrecipitation        0.000498  73222
HazeThunder            0.001250      4
Mist                   0.001741   3018
MistThunder            0.003000      5
UnknownPrecipitation   0.005000      1
LightSnow              0.011849   3101
LightRain              0.021799   4566
Hail                   0.022778      9
Snow                   0.026129    217
Thunder                0.030673    208
Rain                   0.061482   1393
HeavySnow              0.073492     63
LightRainThunder       0.079787    329
RainThunder            0.123646    181
HeavySnowThunder       0.160000      2
HeavyRain              0.207649    285
HeavyRainThunder       0.384223    309


In [137]:
# (2) Do the same for the _is_precip derived flag
diag_df = agg_df.groupby('_is_precip').agg(
    AvgPrecip = pd.NamedAgg(column='Precipitation', aggfunc="mean"),
    Count = pd.NamedAgg(column='Precipitation', aggfunc="count")
    )
print(diag_df.sort_values(by='AvgPrecip'))

            AvgPrecip  Count
_is_precip                  
0            0.000546  76999
1            0.043633  10673


In [138]:
# (3) Finally, correlate the derived CloudCondition with Precipitation
#  (e.g. we expect less precipitation from Clear skies than Clouded ones)

In [139]:
diag_df = agg_df.groupby('CloudCondition').agg(
    AvgPrecip = pd.NamedAgg(column='Precipitation', aggfunc="mean"),
    Count = pd.NamedAgg(column='Precipitation', aggfunc="count")
    )
print(diag_df.sort_values(by='AvgPrecip'))

                AvgPrecip  Count
CloudCondition                  
MostlyClear      0.000021  14546
Clear            0.000109   7705
PartlyCloudy     0.000515  11127
MostlyCloudy     0.001099  20519
Cloudy           0.014100  33518
Obscured         0.022354    257


**Summary So Far**

We have shown how to take a US Governrment report on historic weather data from a single location and convert it to a simpler and cleaner form that can then be used to feed to Machine Learning algorithms implemented in scikit-learn, TensorFlow and the like.

## Some data exploration

Let us now get some idea on what features are the most impactful in predicting the weather outcomes that are of interest:

- `Temp`
- `WindSpeed`
- `_is_precip` (aka "is it going to rain/snow?)
- `_is_clear`

To do that we need to create Feature Sets that:
- combine data for multiple locations (i.e. for 10/10/2015 14:00, include data from Columbus and Cedar Rapids in addition to that from Chicago)
- each row corresponds to a prediction for a certain *Look Ahead Interval*. I.e. , if that interval is 12h, the feature vector for the above example would actually be predicting weather for 10/11/2015 02:00
- encode data preceding the observation timestamp within a certain *Lookback Window* (i.e. for the example above, what was the temperature at 13:00, 12:00 as well as what was the Avg/Max/Min for the last 6h). This is akin to the "Sliding Window" approach

The script that generates such combined files is not trivial and won't be included here. Suffice it to say that it takes as input the output of the code provided above, based on the original US Government weather reports. The source code for the said script can be seen here: https://github.com/sergeypine/WeatherLearner/blob/capstone-data-wrangling/model-prototyping/NOAA_featureset_generator.py 

(The specific geographic locations, look back windows and included fatures are configurable; in this exploration we shall use Feature Set files for a 12h prediction interval that include 9 adjacent locations and result in about 800 features).

If the reader wants to reproduce the results below they shall need to first run the preprocessing script: https://github.com/sergeypine/WeatherLearner/blob/capstone-data-wrangling/data-processing/NOAA_preprocessor.py ; then, they need to run the `NOAA_featureset_generator` script on the output.

In [143]:
# Load the Feature Set for is_precip prediction 12h in advance
df = pd.read_csv('NOAA_featureset__is_precip_LocationCount9_LookBack3h_LookAhead12h.csv')

In [144]:
df.describe() 

,_day_sin_loc0_3h,_day_cos_loc0_3h,_hour_sin_loc0_3h,_hour_cos_loc0_3h,_wind_dir_sin_loc0_3h,_wind_dir_cos_loc0_3h,PressureChange_loc0_3h,_wind_dir_sin_loc1_3h,_wind_dir_cos_loc1_3h,PressureChange_loc1_3h,...,CloudAltitude_loc9_avg_18h,CloudAltitude_loc9_min_18h,CloudAltitude_loc9_max_18h,WindSpeed_loc9_avg_18h,WindSpeed_loc9_min_18h,WindSpeed_loc9_max_18h,WindGust_loc9_avg_18h,WindGust_loc9_min_18h,WindGust_loc9_max_18h,Y
count,8.766000e+04,87660.000000,8.766000e+04,8.766000e+04,87660.000000,8.766000e+04,87660.000000,87660.000000,8.766000e+04,87660.000000,...,87660.000000,87660.000000,87660.000000,87660.000000,87660.000000,87660.000000,87660.000000,87660.000000,87660.000000,87671.000000
mean,1.178190e-05,0.000684,6.992731e-05,7.459569e-05,-0.141420,5.189132e-03,0.000834,-0.044434,-1.483525e-02,0.000153,...,11591.349804,4930.857601,17393.380290,8.448144,3.027707,14.617962,3.536860,0.093018,15.059183,0.255010
std,7.068689e-01,0.707352,7.071042e-01,7.071174e-01,0.670703,7.281088e-01,0.042396,0.660113,7.497121e-01,0.042046,...,6021.810383,6132.572360,5332.125606,3.904595,3.562206,5.115404,5.753863,1.585404,14.426725,0.435869
min,-9.999907e-01,-0.999963,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000e+00,-0.280000,-1.000000,-1.000000e+00,-0.320000,...,0.000000,0.000000,0.000000,0.078947,0.000000,1.500000,0.000000,0.000000,0.000000,0.000000
25%,-7.055836e-01,-0.708627,-7.071068e-01,-7.071068e-01,-0.766044,-7.660444e-01,-0.020000,-0.642788,-8.660254e-01,-0.020000,...,6333.936404,1000.000000,20000.000000,5.640132,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000
50%,6.432491e-16,0.004304,1.224647e-16,6.123234e-17,-0.173648,6.123234e-17,0.000000,0.000000,-1.836970e-16,0.000000,...,12050.657895,2400.000000,20000.000000,7.953509,2.000000,14.000000,1.105263,0.000000,20.000000,0.000000
75%,7.055836e-01,0.702527,7.071068e-01,7.071068e-01,0.500000,7.660444e-01,0.020000,0.500000,7.660444e-01,0.020000,...,16968.421053,5500.000000,20000.000000,10.631579,5.500000,17.250000,4.684211,0.000000,26.000000,1.000000
max,9.999907e-01,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,0.300000,1.000000,1.000000e+00,0.250000,...,20000.000000,20000.000000,20000.000000,30.268484,24.500000,39.000000,43.842105,39.000000,104.000000,1.000000


We can see there are *many* columns. The important points are:

- `Y` is the column we are trying to predict (whether or not it is going to rain/snow/hail/etc in this case)
- `locX` refers to the Geographic Location X where X generally increases from East to West
- `_Nh` refers to how far back in time the variable in the column is for (e.g. Temp_..2h refers to "Temperature 2h ago")
- `avg_nH` refers to the average (or some other aggregation function) over the last `n` hours

(The code below was inspired by this public post: https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b)

In [145]:
# Calculate the correlations
cor = df.corr()

In [150]:
#Correlation with output variable
cor_target = abs(cor["Y"])

#Selecting the (higher) correlated features
relevant_features = cor_target[cor_target>0.25]
relevant_features

_cloud_intensity_loc0_avg_6h     0.260546
_cloud_intensity_loc0_min_6h     0.259554
_cloud_intensity_loc1_avg_6h     0.315719
_cloud_intensity_loc1_min_6h     0.267047
_cloud_intensity_loc1_max_6h     0.282488
CloudAltitude_loc1_avg_6h        0.250359
_cloud_intensity_loc2_avg_6h     0.334898
_cloud_intensity_loc2_min_6h     0.332911
_cloud_intensity_loc2_max_6h     0.266468
Humidity_loc2_min_6h             0.254173
CloudAltitude_loc2_avg_6h        0.305911
CloudAltitude_loc2_min_6h        0.269899
CloudAltitude_loc2_max_6h        0.277564
_cloud_intensity_loc3_avg_6h     0.255726
_cloud_intensity_loc4_avg_6h     0.300967
_cloud_intensity_loc4_min_6h     0.272463
_cloud_intensity_loc4_max_6h     0.254760
CloudAltitude_loc4_avg_6h        0.271504
CloudAltitude_loc4_max_6h        0.252133
_cloud_intensity_loc5_avg_6h     0.274391
_cloud_intensity_loc5_min_6h     0.262439
_cloud_intensity_loc1_avg_18h    0.259900
_cloud_intensity_loc2_avg_18h    0.322531
_cloud_intensity_loc2_min_18h    0

The above does make some intuitive sense! It does stand to reason that cloud characteristics in the target and nearby locations (e.g. `cloud_intensity`) influence the probability of rain.

Furthermore, let's look at what the `X` are for the locations (in the following order):

- Chicago, IL
- Cedar Rapids, IA
- Des Moines, IA
- Quincy, IL
- Rochester, MN
- Madison, WI
- St Louis, MO
- Indianapolis, IN
- Lansing, MI
- Green Bay, WI

Looking at the map, we can see that all the locations that showed up in the list above are to the *West* of our Target location, Chicago. Intriguingly, that follows the popular weather mantra: *Weather Comes from the West*

In [151]:
# Now let's go ahead and go through the same exercise for Temperature:
df = pd.read_csv('NOAA_featureset_Temp_LocationCount9_LookBack3h_LookAhead12h.csv')

# Calculate the correlations
cor = df.corr()

In [164]:
#Selecting the (higher) correlated features
cor_target = abs(cor["Y"])
relevant_features = cor_target[cor_target>0.88]
relevant_features

Temp_loc0_avg_6h         0.888007
Temp_loc0_min_6h         0.891514
Temp_loc0_max_6h         0.886536
DewPoint_loc0_min_6h     0.880464
Temp_loc1_avg_6h         0.884592
Temp_loc1_min_6h         0.891778
Temp_loc1_max_6h         0.881258
DewPoint_loc1_avg_6h     0.908863
DewPoint_loc1_min_6h     0.910106
DewPoint_loc1_max_6h     0.904915
Temp_loc2_avg_6h         0.890565
Temp_loc2_min_6h         0.900608
Temp_loc2_max_6h         0.883506
DewPoint_loc2_avg_6h     0.915690
DewPoint_loc2_min_6h     0.916832
DewPoint_loc2_max_6h     0.911027
Temp_loc3_min_6h         0.882088
DewPoint_loc3_avg_6h     0.889878
DewPoint_loc3_min_6h     0.892749
DewPoint_loc3_max_6h     0.883671
Temp_loc4_avg_6h         0.897467
Temp_loc4_min_6h         0.904123
Temp_loc4_max_6h         0.893422
DewPoint_loc4_avg_6h     0.920915
DewPoint_loc4_min_6h     0.921049
DewPoint_loc4_max_6h     0.918564
Temp_loc5_avg_6h         0.887865
Temp_loc5_min_6h         0.888492
Temp_loc5_max_6h         0.889186
DewPoint_loc5_

This too makes sense! Temperature mostly depends on temperature (and keep in mind that `DewPoint` is temperature too, just with correction for humidity)

Interestingly, the locations are now all over the place, not just *West*

In [171]:
# Just out of curiosity: what determines Temperature the least?
irrelevant_features = cor_target[cor_target < 0.02]
irrelevant_features

_wind_dir_cos_loc8_3h         0.003989
_hour_sin_loc0_2h             0.003763
_wind_dir_cos_loc8_2h         0.007193
_wind_dir_cos_loc8_1h         0.010923
_wind_dir_cos_loc8_0h         0.015121
_is_thunder_loc0_min_6h       0.005368
Humidity_loc0_max_6h          0.004126
_is_thunder_loc1_min_6h       0.006797
_is_thunder_loc2_min_6h       0.009641
Humidity_loc2_min_6h          0.019679
Precipitation_loc3_min_6h     0.015684
_is_thunder_loc3_min_6h       0.017607
_is_thunder_loc4_min_6h       0.008822
Humidity_loc4_avg_6h          0.007353
WindGust_loc4_max_6h          0.005480
_is_thunder_loc5_min_6h       0.009306
Humidity_loc5_min_6h          0.016672
_is_thunder_loc6_min_6h       0.006107
Humidity_loc6_avg_6h          0.012489
Precipitation_loc7_avg_6h     0.007651
_is_thunder_loc7_min_6h       0.005251
Precipitation_loc8_avg_6h     0.012134
_is_thunder_loc2_min_18h      0.000579
Precipitation_loc7_avg_18h    0.017020
Precipitation_loc7_min_18h    0.008539
Humidity_loc7_max_18h    

Looks like moisture-related factors are the least consequential in determining temperature

In [172]:
# Now Wind:
df = pd.read_csv('NOAA_featureset_WindSpeed_LocationCount9_LookBack3h_LookAhead12h.csv')

# Calculate the correlations
cor = df.corr()

In [174]:
#Selecting the (higher) correlated features
cor_target = abs(cor["Y"])
relevant_features = cor_target[cor_target>0.25]
relevant_features

_hour_cos_loc0_3h         0.263384
_hour_cos_loc0_2h         0.287383
_hour_cos_loc0_1h         0.291807
_hour_cos_loc0_0h         0.276353
Pressure_loc1_min_6h      0.251014
WindSpeed_loc1_avg_6h     0.328792
WindSpeed_loc1_min_6h     0.317197
WindSpeed_loc1_max_6h     0.319936
WindGust_loc1_avg_6h      0.254657
WindSpeed_loc2_avg_6h     0.351429
WindSpeed_loc2_min_6h     0.335527
WindSpeed_loc2_max_6h     0.333695
WindGust_loc2_avg_6h      0.296613
WindGust_loc2_max_6h      0.274121
WindSpeed_loc3_avg_6h     0.317881
WindSpeed_loc3_min_6h     0.304735
WindSpeed_loc3_max_6h     0.301558
WindGust_loc3_avg_6h      0.263901
Pressure_loc4_avg_6h      0.253498
Pressure_loc4_min_6h      0.272219
WindSpeed_loc4_avg_6h     0.337683
WindSpeed_loc4_min_6h     0.322325
WindSpeed_loc4_max_6h     0.328462
WindSpeed_loc5_avg_6h     0.255364
WindSpeed_loc6_avg_6h     0.259639
WindSpeed_loc6_max_6h     0.252373
WindSpeed_loc0_avg_18h    0.266848
WindSpeed_loc1_avg_18h    0.349520
WindSpeed_loc1_min_1

Again, we see that only locations in the *West* come up. 

Also, as we might expect, Wind and Pressure (the direct cause of Wind) are the most impactful variables.

Interestingly, time-of-day also appears to be important. 

In [175]:
# Finally, `is_clear` (is the weather going to be sunny-ish (or moon-ish or star-ish))
df = pd.read_csv('NOAA_featureset__is_clear_LocationCount9_LookBack3h_LookAhead12h.csv')

# Calculate the correlations
cor = df.corr()

In [177]:
#Selecting the (higher) correlated features
cor_target = abs(cor["Y"])
relevant_features = cor_target[cor_target>0.2]
relevant_features

_cloud_intensity_loc0_avg_6h     0.280609
_cloud_intensity_loc0_min_6h     0.254778
_cloud_intensity_loc0_max_6h     0.262390
_cloud_intensity_loc1_avg_6h     0.244656
_cloud_intensity_loc1_max_6h     0.245836
Humidity_loc1_min_6h             0.219287
_cloud_intensity_loc2_avg_6h     0.357246
_cloud_intensity_loc2_min_6h     0.297423
_cloud_intensity_loc2_max_6h     0.347290
Humidity_loc2_avg_6h             0.202574
Humidity_loc2_min_6h             0.222186
Pressure_loc2_min_6h             0.200852
_cloud_intensity_loc4_avg_6h     0.281130
_cloud_intensity_loc4_min_6h     0.230645
_cloud_intensity_loc4_max_6h     0.262922
Humidity_loc4_avg_6h             0.211678
Humidity_loc4_min_6h             0.230900
CloudAltitude_loc4_avg_6h        0.251447
CloudAltitude_loc4_min_6h        0.232807
CloudAltitude_loc4_max_6h        0.221594
_cloud_intensity_loc5_avg_6h     0.275285
_cloud_intensity_loc5_min_6h     0.235378
_cloud_intensity_loc5_max_6h     0.258945
Humidity_loc5_min_6h             0

Whether it's going to be clear is, as expected, similar to Precipitation (the two are closely related, after all). Also, the locations are all in the *West*. 